In [21]:

from backend.lamatidb.interfaces.index_interface import IndexInterface
from backend.lamatidb.interfaces.query_interface import QueryInterface
from llama_index.core.vector_stores.types import MetadataFilter, MetadataFilters, FilterOperator
CHAT_WITH_DATA_DB = "test"
CHAT_WITH_DATA_VECTOR_TABLE_NAME = "Ben_full_docs_v3"
cwd_index_interface = IndexInterface(CHAT_WITH_DATA_DB,CHAT_WITH_DATA_VECTOR_TABLE_NAME,embedding_model_name="allenai/scibert_scivocab_uncased")
cwd_index_interface.load_index_from_vector_store()
cwd_index = cwd_index_interface.get_index()

#step 1: retrieve potentially relevant documents
DB_NAME = "test_matias" #os.environ['TIDB_DB_NAME'] # set to test_matias as this has data loaded
VECTOR_TABLE_NAME = "scibert_alldata"


# Load or create the index
index_interface = IndexInterface(DB_NAME, VECTOR_TABLE_NAME,embedding_model_name="allenai/scibert_scivocab_uncased")
index_interface.load_index_from_vector_store()
abstract_index = index_interface.get_index()

def get_query_from_index(query,index,similarity_top_k = 100,filters = None):
    query_interface = QueryInterface(index)
    query_interface.configure_retriever(similarity_top_k=similarity_top_k)
    query_interface.configure_response_synthesizer()
    query_interface.assemble_query_engine()
    if filters is not None:
        response = query_interface.perform_metadata_filtered_query(query, filters)
    else:
        response = query_interface.perform_query(query)
    return response
def get_abstracts_response(query):
    query_interface = QueryInterface(abstract_index)
    query_interface.configure_retriever(similarity_top_k=100)
    query_interface.configure_response_synthesizer()
    query_interface.assemble_query_engine()
    response = query_interface.perform_query(query)
    return response

def chat_with_documents(query):

    abstracts_response = get_abstracts_response(query)
    pmids = [node.metadata['source'] for node in abstracts_response.source_nodes]
    query_interface = QueryInterface(cwd_index)
    query_interface.configure_retriever(similarity_top_k=100)
    query_interface.configure_response_synthesizer()
    query_interface.assemble_query_engine()

    filters = [
    {"key": "PMID", "value": pmids, "operator": FilterOperator.IN},
    ]
    response = query_interface.perform_metadata_filtered_query(query, filters)
    
    if response.source_nodes:
        return (response,f"{str(response)} \n\n Sources:") #{%[node.metadata['PMID'] for node in abstracts_response.source_nodes]}")
    else:
        return (abstracts_response,f"{str(abstracts_response)} \n\n No full papers found. Abstract sources: {pmids}")


No sentence-transformers model found with name allenai/scibert_scivocab_uncased. Creating a new one with mean pooling.
No sentence-transformers model found with name allenai/scibert_scivocab_uncased. Creating a new one with mean pooling.


Time taken to load the index: 0.00 seconds
Time taken to load the index: 0.00 seconds


In [11]:
response, text = chat_with_documents('women with substance abuse')
print(text)

The relationship between traumatic stress symptoms and substance abuse has been well established in the literature. In a study focusing on young adults, it was found that traumatic stress symptoms, including dissociative behavior, intrusive thoughts, and low levels of tension reduction behaviors, were significantly associated with alcohol use. This highlights the importance of understanding and addressing traumatic stress symptoms in women with substance abuse issues. 

 No full papers found. Abstract sources: ['16633531', '16626877', '16627079', '16629084', '16631925', '16629521', '16629083', '16633458', '16632210', '16630198']


In [13]:
response, text = chat_with_documents('leading cause of death')
print(text)

Stroke is the leading cause of long-term disability worldwide. 

 No full papers found. Abstract sources: ['16625951', '16631716', '16629437', '16633479', '16625944', '16626917', '16627089', '16633507', '16629536', '16632652']


In [22]:
response, text = chat_with_documents('Vascular calcification (VC) is concomitant with atherosclerosis. How do CD163+ macrophages inhibit VC ')
print(text)

CD163+ macrophages inhibit vascular calcification by promoting the clearance of hemoglobin and heme, which are known to accelerate calcification. 

 Sources:


In [24]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='8e4ab8da-3c0b-4696-b76a-29990f050f9c', embedding=None, metadata={'PMCID': 'PMC7114336', 'PMID': '16626815'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b792b309-9acf-496e-b626-10b382dc82c2', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'PMCID': 'PMC7114336', 'PMID': '16626815'}, hash='d8dc97bc7e3f3004871d29844dc1b86e93e717c7580126de40a881f708198dde'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='0561737c-1112-42ed-81bc-0a629d463662', node_type=<ObjectType.TEXT: '1'>, metadata={'PMCID': 'PMC7114336', 'PMID': '16626815'}, hash='7310b13bace838f248a1a39e58c3c8489812d52f115a23aecbc6c7d3ab20e9f8'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8ef58075-218f-4c92-a2be-1475d0fb3c0b', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='194787ffa76ac8859de893f167a7e6b239a1472a14f8977c2a1552d725f25e1e')}, text="Activation of MPC results from sti

In [18]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='cf4329f2-8df9-43d4-884a-26d92da90afa', embedding=None, metadata={'source': '16632376', 'title': 'Cellular inflammatory responses: novel insights for obesity and insulin resistance.', 'authors': 'Hong Chen', 'year': '2006'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d01309c8-8a8a-40c5-9944-472d4972f64c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'source': '16632376', 'title': 'Cellular inflammatory responses: novel insights for obesity and insulin resistance.', 'authors': 'Hong Chen', 'year': '2006'}, hash='5f1290cd1bda4c6a6cf0b5c593448b72752f5286e18ebfac2172cda73a368f7b')}, text='Type 2 diabetes is rapidly becoming a worldwide epidemic. Obesity and sedentary lifestyle are the main environmental causes for the development of insulin resistance and type 2 diabetes. In the past decade, it has been increasingly recognized that obesity and insulin resistance 

In [1]:
import streamlit_app

c:\Users\Work\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\Local\pypoetry\Cache\virtualenvs\lamatidb-gXYSf88G-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-19 22:19:32.650 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
No sentence-transformers model found with name allenai/scibert_scivocab_uncased. Creating a new one with mean pooling.


Time taken to load the index: 0.82 seconds


2024-08-19 22:19:36.298 
  command:

    streamlit run c:\Users\Work\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\Local\pypoetry\Cache\virtualenvs\lamatidb-gXYSf88G-py3.11\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
import os
import streamlit.web.bootstrap
import streamlit.config as _config

_config.set_option("server.headless", True)
args = []

streamlit.web.bootstrap.run(r'C:\Users\Work\pico-scholar\backend\lamatidb\interfaces\temp_streamlit_app.py',False,args,{})

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
with open("pmid.txt", "r") as file:
    type(file.read())